In [1]:
# rendering related
import os
import imageio
import jax
import mujoco
from dm_control.mujoco import wrapper
from dm_control.mujoco.wrapper.mjbindings import enums
import pickle
from tqdm import tqdm

ModuleNotFoundError: No module named 'imageio'

In [2]:
# loading data
with open("./../clips/all_snips.p", "rb") as f:
    all_traj = pickle.load(f)

# render overlay
scene_option = wrapper.MjvOption()
scene_option.geomgroup[2] = 1
scene_option.sitegroup[2] = 1

scene_option.sitegroup[3] = 1
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = False
scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
scene_option.flags[enums.mjtRndFlag.mjRND_SHADOW] = False
scene_option.flags[enums.mjtRndFlag.mjRND_REFLECTION] = False
scene_option.flags[enums.mjtRndFlag.mjRND_SKYBOX] = False
scene_option.flags[enums.mjtRndFlag.mjRND_FOG] = False
mj_model = mujoco.MjModel.from_xml_path(f"../assets/rodent.xml")

mj_model.opt.solver = {
    "cg": mujoco.mjtSolver.mjSOL_CG,
    "newton": mujoco.mjtSolver.mjSOL_NEWTON,
}["cg"]

mj_model.opt.iterations = 6
mj_model.opt.ls_iterations = 6
mj_model.opt.jacobian = 0  # dense
mj_data = mujoco.MjData(mj_model)

os.environ["MUJOCO_GL"] = "egl"

renderer = mujoco.Renderer(mj_model, height=512, width=512)
mujoco.mj_kinematics(mj_model, mj_data)

In [3]:
len(all_traj["snips_order"])

842

In [4]:
all_traj["qpos"].shape

(210500, 74)

In [5]:
qpos_clips = all_traj["qpos"].reshape((-1, 250, 74))

In [6]:
all_traj["snips_order"]

['././snippets_2_25_2021/snips/FastWalk_171.p',
 '././snippets_2_25_2021/snips/LGroom_158.p',
 '././snippets_2_25_2021/snips/FastWalk_169.p',
 '././snippets_2_25_2021/snips/FastWalk_44.p',
 '././snippets_2_25_2021/snips/RGroom_45.p',
 '././snippets_2_25_2021/snips/FastWalk_58.p',
 '././snippets_2_25_2021/snips/Rear_155.p',
 '././snippets_2_25_2021/snips/RGroom_16.p',
 '././snippets_2_25_2021/snips/Rear_82.p',
 '././snippets_2_25_2021/snips/Walk_145.p',
 '././snippets_2_25_2021/snips/Rear_175.p',
 '././snippets_2_25_2021/snips/RGroom_164.p',
 '././snippets_2_25_2021/snips/Walk_115.p',
 '././snippets_2_25_2021/snips/Walk_98.p',
 '././snippets_2_25_2021/snips/Rear_104.p',
 '././snippets_2_25_2021/snips/Rear_166.p',
 '././snippets_2_25_2021/snips/Walk_174.p',
 '././snippets_2_25_2021/snips/Rear_158.p',
 '././snippets_2_25_2021/snips/LGroom_134.p',
 '././snippets_2_25_2021/snips/Rear_5.p',
 '././snippets_2_25_2021/snips/Rear_135.p',
 '././snippets_2_25_2021/snips/FastWalk_87.p',
 '././snipp

In [7]:
qpos_clips

Array([[[ 0.3634157 ,  0.17606783,  0.15217969, ..., -0.05047335,
         -0.09988861,  0.        ],
        [ 0.36408123,  0.17538573,  0.15383212, ..., -0.20306881,
         -0.13130334,  0.        ],
        [ 0.36483166,  0.17535505,  0.15488204, ..., -0.28572166,
         -0.13841768,  0.        ],
        ...,
        [ 0.03164212,  0.15086763,  0.06442218, ..., -0.27511814,
         -0.17720379,  0.        ],
        [ 0.02865947,  0.15237886,  0.06489865, ..., -0.24613251,
         -0.1861186 ,  0.        ],
        [ 0.02535689,  0.15474537,  0.06531984, ..., -0.1727609 ,
         -0.190735  ,  0.        ]],

       [[-0.20612271,  0.09377342,  0.07394315, ...,  0.19856702,
         -0.02486288,  0.        ],
        [-0.2064968 ,  0.09413765,  0.07397661, ...,  0.32640502,
         -0.05202457,  0.        ],
        [-0.20690076,  0.0937114 ,  0.07379968, ...,  0.39135835,
         -0.07478626,  0.        ],
        ...,
        [-0.20320179,  0.10944644,  0.07677754, ...,  

In [8]:
all_traj["snips_order"][0].split("/")[-1].split(".")[0]

'FastWalk_171'

In [11]:
for i, name in enumerate(tqdm(all_traj["snips_order"])):
    frames = []
    # render while stepping using mujoco
    name = name.split("/")[-1].split(".")[0]
    video_path = f"videos/{name}.mp4"
    with imageio.get_writer(video_path, fps=50) as video:
        for qpos in tqdm(qpos_clips[i], leave=False):
            # Set keypoints
            mj_data.qpos = qpos
            mujoco.mj_forward(mj_model, mj_data)
            renderer.update_scene(mj_data, camera="close_profile")
            pixels = renderer.render()
            video.append_data(pixels)
            frames.append(pixels)

  0%|          | 0/842 [00:00<?, ?it/s]


TypeError: write_frames() got an unexpected keyword argument 'audio_path'

# Good Clip to Imitate

Access directly by names from `all_snips.p`

- FastWalk_61
- FastWalk_65
- FastWalk_80
- FastWalk_93
- Walk_20
- Walk_27
- Walk_34
- Walk_49